In [1]:
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch

import warnings

warnings.filterwarnings(action='ignore')
torch.set_printoptions(sci_mode=True)

DATA_PATH = '/opt/ml/final-project-level3-recsys-05/Model/Model-Experiment/data'
MODEL_PATH = '/opt/ml/final-project-level3-recsys-05/Model/Model-Experiment/model'
VAL_TO_IDX_DATA_PATH = '/opt/ml/final-project-level3-recsys-05/Model/Model-Server/data'

PICKLE_PROTOCOL = 4

In [2]:
class EASE():
    def __init__(self, reg):
        self.reg = reg
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    def clear_memory(self):
        gc.collect()
        torch.cuda.empty_cache()

    def fit(self, X):
        X = X.to(self.device)
        G = X.t() @ X
        diagIndices = torch.eye(G.shape[0]) == 1
        G[diagIndices] += self.reg

        P = G.inverse()
        B = P / (-1 * P.diag())
        B[diagIndices] = 0

        self.B = B.cpu()
    
    def predict(self, X):
        output = (X.to(self.device) @ self.B.to(self.device)).cpu()
        return output

def get_hit(pred_list, true_list):
    hit_list = set(true_list) & set(pred_list)
    hit = len(hit_list) / len(true_list)
    return hit

In [3]:
import gc

gc.collect()
torch.cuda.empty_cache()

In [4]:
import json

with open(os.path.join(VAL_TO_IDX_DATA_PATH, 'clean_problem_id_to_idx.json'), 'r', encoding = 'utf-8') as f:
    problemId_to_idx = json.load(f)

with open(os.path.join(VAL_TO_IDX_DATA_PATH, 'clean_idx_to_problem_id.json'), 'r', encoding = 'utf-8') as f:
    idx_to_problemId = json.load(f)

In [5]:
user_df = pd.read_csv(os.path.join(DATA_PATH, 'user.csv'))

In [6]:
def get_problems_to_idx(problems):
    problems = eval(problems)
    ret = []
    for problemId in problems:
        try: ret.append(int(problemId_to_idx[problemId]))
        except: continue
    return ret

user_df['problems_to_idx'] = user_df['problems'].apply(lambda x : get_problems_to_idx(x))

In [7]:
new_user_df = user_df[user_df['problems_to_idx'].apply(lambda x : True if len(x) >= 15 else False)].reset_index(drop = False)

In [8]:
user_name_to_idx = {}

for idx, user_name in enumerate(new_user_df['user_name'].tolist()):
    user_name_to_idx[user_name] = idx

# 평가

In [10]:
mat = torch.zeros(size = (len(user_name_to_idx), len(problemId_to_idx)))
valid_dict = {}

group_df = new_user_df.groupby('user_name')

for user_name, df in group_df:
    random.seed(22)
    total = df['problems_to_idx'].values[0]
    valid = random.sample(total, 10)
    train = list(set(total) - set(valid))
    
    mat[user_name_to_idx[user_name], train] = 1

    valid_dict[user_name_to_idx[user_name]] = valid

In [11]:
for reg in tqdm([100000, 10000, 1000, 100, 10, 1]):
    model = EASE(reg = reg)
    model.fit(mat)
    model.clear_memory()

    output = model.predict(mat)
    model.clear_memory()

    output[mat == 1] = -np.Inf
    rec_list = output.argsort(dim = 1)

    hit = 0
    for idx, rec in enumerate(rec_list):
        pred = rec.cpu().numpy().tolist()[::-1][:10]
        true = valid_dict[idx]
        hit += get_hit(pred, true)

    hit /= len(output)
    print(f'reg: {reg} | hit : {hit:.5f}')

 17%|█▋        | 1/6 [00:36<03:03, 36.63s/it]

reg: 100000 | hit : 0.44241


 33%|███▎      | 2/6 [01:08<02:20, 35.21s/it]

reg: 10000 | hit : 0.52588


 50%|█████     | 3/6 [01:40<01:42, 34.23s/it]

reg: 1000 | hit : 0.54956


 67%|██████▋   | 4/6 [02:12<01:07, 33.55s/it]

reg: 100 | hit : 0.54771


 83%|████████▎ | 5/6 [02:44<00:33, 33.06s/it]

reg: 10 | hit : 0.53965


# 예측

In [9]:
mat = torch.zeros(size = (len(user_name_to_idx), len(problemId_to_idx)))

group_df = new_user_df.groupby('user_name')

for user_name, df in group_df:
    total = df['problems_to_idx'].values[0]
    mat[user_name_to_idx[user_name], total] = 1

In [10]:
reg = 1000
model = EASE(reg = reg)
model.fit(mat)
model.clear_memory()

In [11]:
import pickle

with open(os.path.join(MODEL_PATH, 'clean-ease.pickle'), 'wb') as file:
    pickle.dump(model, file, protocol = PICKLE_PROTOCOL)